In [50]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy
from sklearn.metrics import roc_auc_score
from sklearn import mixture, preprocessing, datasets

from importlib import reload
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import patches

import torch.utils.data as data_utils

import utils.models as models
import utils.plotting as plotting
import utils.dataloaders as dl
import utils.traintest as tt
import utils.adversarial as adv
import utils.eval as ev
import utils.resnet_orig as resnet
import utils.gmm_helpers as gmm_helpers

import model_paths

from tqdm import tqdm

import utils.single_maha as maha

import model_params

reload(maha)

<module 'utils.single_maha' from '/home/alexm/project/notebooks/gmm-robust/utils/single_maha.py'>

In [ ]:
dataset = 'CIFAR10'

batch = torch.load('results/backup/' + file)

model_params = params.params_dict[dataset]()
model_path = model_paths.model_dict[dataset]() 
model_list = [torch.load(file).cpu().eval() for file in list(model_path.file_dict.values())[:-1]]

In [51]:
device = torch.device('cuda:6')
model = torch.load('SavedModels/base/base_CIFAR10_lr0.1_augm_flagTrue_train_typeplain.pth').to(device)
model = model.eval()

In [52]:
base_model = maha.ResNet18(num_of_channels=3, num_classes=10)

In [53]:
base_model.load_state_dict(model.state_dict())
base_model = base_model.eval().to(device)

In [54]:
params = model_params.params_dict['CIFAR10']()

In [55]:
maha_model = maha.Mahalanobis(base_model, params, device)

In [56]:
maha_model.to(device)

Mahalanobis(
  (model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (

In [63]:
reload(maha)
ODIN_model = maha.ModelODIN(maha_model, params, device)

In [73]:
data_out = next(iter(params.tinyimage_loader))[0].to(device)

In [95]:
out1 = ODIN_model(data).max(1)[0]
out2 = ODIN_model(data_out).max(1)[0]

In [96]:
(out1[:,None]>out2[None,:]).float().mean()

tensor(0.9463, device='cuda:6')

In [94]:
ODIN_model.epsilon= 0.0005